# Scorer Notebook
The main notebook to do postprocessing and score the results. Postprocessing can be run via CLI or through this notebook

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import json
import os
import gc
import warnings
from typing import List, Dict, Any
import time
import torch
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from vllm import LLM, SamplingParams
from utils import postprocessing, scoring_function

sys.path.insert(0, "../src/")
sys.path.insert(0, "..")
from config import REPO_ROOT

INFO 07-14 09:51:35 [__init__.py:239] Automatically detected platform cuda.


## Postprocessing

This might already be done in the bash script

In [2]:
# iterature through files and run postprocessing
standard_filepath = REPO_ROOT / "results/heart_disease_minimal/claude_3_7_sonnet.json" # o3, claude_3_7_sonnet. gpt4_1
results = postprocessing(standard_filepath, force=True, distance_metric="gower", save=True, explicit_dataset = "")

Extracted Dataset Name: heart_disease_minimal


  0%|          | 0/1936 [00:00<?, ?it/s]

In [4]:
datasets = ["income", "income_minimal", "house_prices", "house_prices_minimal", "heart_disease", "heart_disease_minimal"]
for dataset in datasets:
    standard_filepath = REPO_ROOT / f"analysis/temperature_1/{dataset}/claude_3_7_sonnet.json" # o3, claude_3_7_sonnet. gpt4_1
    results = postprocessing(standard_filepath, force=True, distance_metric="gower", save=True, explicit_dataset = "")

Extracted Dataset Name: income


  0%|          | 0/1920 [00:00<?, ?it/s]

Extracted Dataset Name: income_minimal


  0%|          | 0/1920 [00:00<?, ?it/s]

Extracted Dataset Name: house_prices


...

  0%|          | 0/1600 [00:00<?, ?it/s]

Extracted Dataset Name: heart_disease


  0%|          | 0/1936 [00:00<?, ?it/s]

Extracted Dataset Name: heart_disease_minimal


  0%|          | 0/1936 [00:00<?, ?it/s]

## Scoring

Score the results for specific models and datasets

In [3]:
scores = scoring_function(standard_filepath, verbose=1, overwrite=False)
print(scores)



----------------------------------------------------------------------------------------------------
Generation Errors + Warnings:
Original Answer Errors:		 0
Counterfactual Errors:		 0
Validation Answer Errors:	 46
Overall Errors (dropped):	 46
Short counterfactuals:		 46. [2, 51, 89, 275, 352, 527, 561, 614, 702, 845, 876, 878, 1029, 1054, 1056, 1118, 1141, 1201, 1218, 1223, 1305, 1384, 1471, 1482, 1539, 1544, 1569, 1577, 1658, 1668, 1671, 1681, 1709, 1731, 1739, 1745, 1809, 1820, 1835, 1845, 1887, 1888, 1908, 1910, 1919, 1922]

Key data:
Filepath:				 /home/kebl7418/projects/counterfactuals/results/heart_disease_minimal/o3.json
Sample size:			 1890
Validity:			 31.75%

Distance and Minimality:
Mean intervention distance:	 0.0459
Mean RCF:			 0.6397
Median RCF:			 0.0000
Mean ICF:			 0.7941
Median ICF:			 0.7143
Total Minimality:		 0.4875

Absolutes:
Mean RCF_A:			 0.0210
Median RCF_A:			 0.0000
Mean ICF_A:			 0.0539
Median ICF_A:			 0.0500
Total Minimality_A:		 0.0435
------------

#### Summary stats

In [4]:
"""
Compute the mean and standard deviation (population and sample)
for the exact-match percentages reported in Table 2.
"""

# Table values: (Income, House prices, Heart disease) for each model
values = [
    0.00, 0.00, 0.15,   # Gemma 2 27B
    3.33, 0.00, 0.00,   # Llama 3 3 70B
    6.89, 0.06, 0.05,   # DeepSeek-R1 32B
    19.70, 0.19, 0.42,  # DeepSeek-R1 70B
    9.95, 0.00, 0.00,   # Claude Sonnet 3.7
    15.70, 0.00, 0.00,  # GPT-4.1
    4.27, 0.00, 0.00    # o3
]

# Convert to NumPy array (optional but convenient)
arr = np.array(values)

mean_val = np.mean(arr)
std_population = np.std(arr, ddof=0)  # Population SD (N)
std_sample = np.std(arr, ddof=1)      # Sample SD (N-1)

print(f"Values (n={len(arr)}):\n{arr}\n")
print(f"Mean                 : {mean_val:.6f}")
print(f"Population SD (σ)    : {std_population:.6f}")
print(f"Sample SD (s, unbiased): {std_sample:.6f}")


Values (n=21):
[ 0.    0.    0.15  3.33  0.    0.    6.89  0.06  0.05 19.7   0.19  0.42
  9.95  0.    0.   15.7   0.    0.    4.27  0.    0.  ]

Mean                 : 2.890952
Population SD (σ)    : 5.499384
Sample SD (s, unbiased): 5.635192


#### o3

In [5]:
456 # number of times it selected the max
998 # number of times it originally predicted below
# 46% 0.4569
456/998

# below...

# 298, 930 #0.32043
# 357, 1006
(298+357)/(930+1006) # 33.83
 

0.33832644628099173